In [ ]:
# AI Agent Workflow 基础教程

## 🎯 学习目标（预计18分钟）
- **理解 AI Agent 核心架构模式** (Workflow vs Agent)
- **掌握银行场景下的智能代理设计** (资讯摘要 + 交易趋势)
- **学会构建简单、透明、稳健的 Agent 系统**
- **实践多智能体协作模式** (Orchestrator-Workers)

## 📊 课程结构
- **前言与概念** (3分钟): Agent vs Workflow 关键区别
- **核心模式解析** (5分钟): 五大设计模式实战应用
- **资讯摘要 Workflow** (4分钟): Prompt Chaining 实现
- **交易分析 Agent** (4分钟): 自主规划与工具调用
- **多智能体协作** (2分钟): Orchestrator-Workers 银行应用

## ⚠️ 合规声明
**本教程所有投资分析内容仅供学习参考，不构成投资建议。银行实际应用中，所有交易决策必须经过严格的风控审批流程。**

---

### 基于 Anthropic 官方最佳实践
> 参考资料：[Building Effective Agents](https://www.anthropic.com/engineering/building-effective-agents)


In [1]:
# 环境设置和核心库导入

# 导入Python内置库
import sqlite3  # Python内置数据库
import os
import json
import time
import random
from datetime import datetime
from typing import List, Dict, Any

print("🔧 检查和安装依赖包...")

# 安装Anthropic API
try:
    import anthropic
    print("   ✅ Anthropic API 已可用")
except ImportError:
    print("   ⚠️ 正在安装 anthropic...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "anthropic", "--quiet"])
    import anthropic
    print("   ✅ Anthropic API 安装完成")

print("\n📦 核心功能状态：")
print("   🗄️ SQLite本地存储 ✅ (Python内置)")
print("   🔧 JSON数据处理 ✅")
print("   🎲 模拟数据生成 ✅")
print("   📅 时间处理功能 ✅")
print("   🤖 Claude API ✅")
print("\n🚀 环境准备完成，开始AI Agent教程！")


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
✅ 环境依赖已安装完成
📦 核心库：Anthropic Claude API, SQLite本地存储, 网络请求工具


In [ ]:
# Claude API 配置和调用函数

# API配置
api_key = os.getenv("ANTHROPIC_API_KEY")
if not api_key:
    raise ValueError("❌ 请设置 ANTHROPIC_API_KEY 环境变量\n💡 设置方法: export ANTHROPIC_API_KEY='your-api-key'")

# 创建客户端
client = anthropic.Anthropic(api_key=api_key)
print("✅ Claude API 客户端创建成功")

def call_claude_stream(prompt: str, system: str = "", max_tokens: int = 1000, temperature: float = 0.7) -> str:
    """
    Claude API调用函数（银行AI Agent核心）
    
    参数:
    - prompt: 用户输入的提示词
    - system: 系统提示词
    - max_tokens: 最大token数
    - temperature: 随机性参数（0-1，银行场景建议0.3-0.7）
    """
    full_response = ""
    try:
        with client.messages.stream(
            model="claude-3-5-sonnet-20241022",
            max_tokens=max_tokens,
            temperature=temperature,
            system=system,
            messages=[{"role": "user", "content": prompt}]
        ) as stream:
            for text in stream.text_stream:
                print(text, end="", flush=True)
                full_response += text
    except Exception as e:
        print(f"❌ API调用失败: {e}")
        raise
    
    print()  # 添加换行
    return full_response

print("🔧 Claude API 配置完成")
print("🚀 准备执行银行AI Agent教程")


In [ ]:
## 📋 AI Agent 核心概念解析（3分钟）

### Workflow vs Agent：Anthropic 官方定义

| 特征 | **Workflow** | **Agent** |
|------|-------------|-----------|
| **控制方式** | 预定义代码路径 | LLM动态自主规划 |
| **适用场景** | 固定步骤、可预测任务 | 开放式、步骤难预判 |
| **银行应用** | 晨报生成、合规检查 | 复杂投研分析、客户问题处理 |
| **成本延迟** | 低成本、快速响应 | 高成本、深度分析 |

### 🏦 银行业务中的选择策略

**⭐ 核心原则：寻找最简单的解决方案**

1. **单次LLM调用** + 检索：60%的银行查询场景
2. **Workflow模式**：标准化流程（如资讯摘要）
3. **Agent模式**：复杂决策（如投资组合分析）

### 🔧 三大设计原则（Anthropic推荐）

1. **Simplicity 简洁优先**：能用单步解决就别用多步
2. **Transparency 透明可诊断**：显式输出思考过程
3. **Robust ACI 稳健接口**：工具描述要"傻瓜化"

> **ACI (Agent-Computer Interface)** = 像设计用户界面一样精心设计Agent-工具接口


In [ ]:
# SQLite 本地记忆存储系统初始化
def init_memory_db():
    """初始化SQLite数据库用于Agent记忆存储"""
    conn = sqlite3.connect('bank_agent_memory.db')
    cursor = conn.cursor()
    
    # 创建资讯存储表
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS news_analysis (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT NOT NULL,
            content TEXT,
            summary TEXT,
            sentiment TEXT,
            risk_level TEXT,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    # 创建交易分析表
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS trading_analysis (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            symbol TEXT NOT NULL,
            analysis_type TEXT,
            indicators TEXT,
            recommendation TEXT,
            confidence_score REAL,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    conn.commit()
    print("📊 银行AI记忆数据库初始化完成")
    print("🗃️  包含表: news_analysis, trading_analysis")
    return conn

# 初始化数据库
db_conn = init_memory_db()


In [ ]:
## 🔗 五大Agentic设计模式解析（5分钟）

基于 Anthropic 官方指南，以下是生产环境中最常见的设计模式：

### 1. **Prompt Chaining 串行链式**
- **机制**：任务分解为固定步骤序列
- **银行应用**：资讯摘要 → 风险评估 → 合规审查
- **优势**：可控、可预测、易调试

### 2. **Routing 智能路由**
- **机制**：分类输入，导向专门的子流程
- **银行应用**：客户问题分类（行情查询/政策解读/风险咨询）
- **优势**：专业化处理，避免通用模型性能折衷

### 3. **Parallelization 并行处理**
- **Sectioning**：独立子任务并行执行
- **Voting**：同一任务多次执行投票
- **银行应用**：多指标并行计算（RSI + MACD + KDJ）

### 4. **Orchestrator-Workers 编排-执行**
- **机制**：中央LLM动态分解任务，委托给专门工作者
- **银行应用**：复杂投研报告（基本面+技术面+风险面）
- **优势**：灵活适应不可预测的复杂任务

### 5. **Evaluator-Optimizer 评估-优化**
- **机制**：生成-评审循环，迭代改进
- **银行应用**：投资建议书撰写与风险措辞优化
- **优势**：质量控制，确保输出符合金融合规要求

> **⚠️ 选择策略**：优先考虑简单模式，只有在必要时才增加复杂性


In [ ]:
# 实战示例1：资讯摘要 Workflow（Prompt Chaining模式）

class NewsAnalysisWorkflow:
    """资讯分析工作流 - 基于Prompt Chaining模式"""
    
    def __init__(self, db_connection):
        self.db = db_connection
    
    def step1_extract_info(self, news_content: str) -> Dict:
        """步骤1：信息提取"""
        system_prompt = """你是银行资讯分析专家。从新闻中提取关键信息，输出JSON格式：
        {
            "title": "新闻标题",
            "key_points": ["要点1", "要点2"],
            "mentioned_stocks": ["股票代码"],
            "policy_impact": "政策影响描述"
        }"""
        
        prompt = f"请分析以下财经新闻：\n\n{news_content}"
        
        print("🔍 步骤1：信息提取中...")
        response = call_claude_stream(prompt, system_prompt, max_tokens=500, temperature=0.3)
        
        try:
            # 提取JSON部分
            start = response.find('{')
            end = response.rfind('}') + 1
            return json.loads(response[start:end])
        except:
            return {"title": "解析失败", "key_points": [], "mentioned_stocks": [], "policy_impact": ""}
    
    def step2_generate_summary(self, extracted_info: Dict) -> str:
        """步骤2：生成摘要"""
        system_prompt = """你是银行投资顾问，为客户撰写简洁的投资资讯摘要。
        要求：
        1. 100字以内
        2. 突出投资相关性
        3. 语言专业但易懂"""
        
        prompt = f"""基于提取的信息生成摘要：
        标题：{extracted_info.get('title', '')}
        要点：{extracted_info.get('key_points', [])}
        涉及股票：{extracted_info.get('mentioned_stocks', [])}
        政策影响：{extracted_info.get('policy_impact', '')}"""
        
        print("📝 步骤2：摘要生成中...")
        return call_claude_stream(prompt, system_prompt, max_tokens=300, temperature=0.5)

# 模拟财经新闻数据
sample_news = """
央行宣布降准0.5个百分点释放长期资金约1万亿元

中国人民银行今日宣布，为维护银行体系流动性合理充裕，决定于2024年2月5日下调金融机构存款准备金率0.5个百分点（不含已执行5%存款准备金率的金融机构）。此次降准释放长期资金约1万亿元。

央行表示，此次降准旨在保持流动性合理充裕，支持实体经济发展。降准后，银行业金融机构加权平均存款准备金率约为7.0%。市场人士认为，此举将有利于银行体系流动性，支持银行加大对实体经济的支持力度，特别是对小微企业和民营企业的支持。

受此消息影响，银行板块集体上涨，工商银行、建设银行、农业银行等大型银行股价均有不同程度上涨。
"""

# 执行工作流
workflow = NewsAnalysisWorkflow(db_conn)
print("🚀 开始执行资讯分析工作流\n" + "="*50)


In [ ]:
# 执行工作流步骤1-2并添加合规检查
extracted_data = workflow.step1_extract_info(sample_news)
print(f"\n✅ 提取结果：{json.dumps(extracted_data, ensure_ascii=False, indent=2)}\n")

summary = workflow.step2_generate_summary(extracted_data)
print(f"\n📋 生成摘要完成\n")

# 步骤3：合规检查（Evaluator模式）
def step3_compliance_check(summary: str) -> str:
    """步骤3：合规性检查和风险提示"""
    system_prompt = """你是银行合规官员，审查投资资讯是否符合监管要求。
    检查内容：
    1. 是否包含明确的风险提示
    2. 是否避免直接投资建议
    3. 是否符合金融机构信息披露规范
    
    输出格式：审查通过的最终版本+必要的免责声明"""
    
    prompt = f"请审查以下投资资讯摘要的合规性：\n\n{summary}"
    
    print("⚖️ 步骤3：合规检查中...")
    return call_claude_stream(prompt, system_prompt, max_tokens=400, temperature=0.2)

final_summary = step3_compliance_check(summary)
print(f"\n✅ 工作流执行完成！")

# 保存到数据库
cursor = db_conn.cursor()
cursor.execute('''
    INSERT INTO news_analysis (title, content, summary, sentiment, risk_level)
    VALUES (?, ?, ?, ?, ?)
''', (
    extracted_data.get('title', ''),
    sample_news[:500],  # 截取前500字符
    final_summary,
    'positive',  # 可以通过额外的分析确定
    'medium'
))
db_conn.commit()
print("💾 分析结果已保存到本地数据库")


In [ ]:
## 🤖 实战示例2：交易趋势分析 Agent（4分钟）

### Agent vs Workflow 的关键区别

**刚才的资讯摘要**是 **Workflow**：
- ✅ 固定3步骤：提取 → 摘要 → 合规
- ✅ 路径可预测，适合标准化处理

**接下来的交易分析**是 **Agent**：
- 🧠 LLM自主决定分析深度和工具选择
- 🔄 基于数据反馈动态调整策略
- 🎯 适合复杂的、步骤不可预测的分析任务

### 🛠️ 工具定义：关键在于"傻瓜化"描述

根据 Anthropic 建议，工具接口设计要像人机界面一样精心设计：
- **参数名称直观**：`stock_symbol` 而不是 `sym`
- **完整示例**：覆盖边界情况
- **格式校验**：防止常见错误


In [ ]:
# 定义交易分析Agent的工具集

class TradingAnalysisTools:
    """交易分析工具集 - 遵循Anthropic ACI最佳实践"""
    
    @staticmethod
    def get_stock_data(stock_symbol: str, days: int = 30) -> Dict:
        """
        获取股票历史数据
        
        参数说明：
        - stock_symbol: 股票代码，格式如 "000001.SZ" (平安银行)
        - days: 获取天数，建议1-365天
        
        返回：包含价格、成交量等数据的字典
        
        示例用法：get_stock_data("000001.SZ", 30)
        """
        # 模拟数据 - 实际应用中接入真实数据源
        import random
        base_price = 15.0
        data = []
        
        for i in range(days):
            price = base_price + random.uniform(-0.5, 0.5)
            volume = random.randint(8000000, 15000000)
            data.append({
                "date": f"2024-01-{i+1:02d}",
                "open": round(price, 2),
                "high": round(price + random.uniform(0, 0.3), 2),
                "low": round(price - random.uniform(0, 0.3), 2),
                "close": round(price, 2),
                "volume": volume
            })
            base_price = price
        
        return {
            "symbol": stock_symbol,
            "data": data[-10:],  # 返回最近10天数据
            "current_price": data[-1]["close"],
            "avg_volume": sum([d["volume"] for d in data]) // days
        }
    
    @staticmethod
    def calculate_technical_indicators(stock_data: Dict) -> Dict:
        """
        计算技术指标
        
        参数：get_stock_data返回的数据字典
        返回：RSI, MACD, 移动平均线等技术指标
        """
        prices = [float(d["close"]) for d in stock_data["data"]]
        volumes = [int(d["volume"]) for d in stock_data["data"]]
        
        # 简化的技术指标计算
        sma_5 = sum(prices[-5:]) / 5 if len(prices) >= 5 else prices[-1]
        sma_10 = sum(prices[-10:]) / 10 if len(prices) >= 10 else prices[-1]
        
        # 模拟RSI计算
        rsi = 45 + random.uniform(-20, 20)
        
        return {
            "RSI": round(rsi, 2),
            "SMA_5": round(sma_5, 2),
            "SMA_10": round(sma_10, 2),
            "current_price": prices[-1],
            "volume_trend": "上升" if volumes[-1] > volumes[-2] else "下降",
            "price_momentum": "看涨" if prices[-1] > sma_5 else "看跌"
        }
    
    @staticmethod
    def get_market_sentiment(stock_symbol: str) -> Dict:
        """
        获取市场情绪数据
        
        参数：股票代码
        返回：舆情分析、机构评级等信息
        """
        # 模拟市场情绪数据
        sentiments = ["积极", "中性", "谨慎"]
        ratings = ["买入", "持有", "卖出"]
        
        return {
            "market_sentiment": random.choice(sentiments),
            "institutional_rating": random.choice(ratings),
            "news_sentiment_score": round(random.uniform(0.3, 0.8), 2),
            "analyst_target_price": round(15.0 + random.uniform(-2, 3), 2)
        }

# 测试工具集
tools = TradingAnalysisTools()
print("🔧 交易分析工具集已初始化")
print("📊 可用工具：股票数据获取、技术指标计算、市场情绪分析")


In [ ]:
# 实现自主交易分析Agent

class TradingAnalysisAgent:
    """
    交易趋势分析智能体 - 基于Anthropic Agent模式
    特点：自主规划、动态工具选择、环境反馈驱动
    """
    
    def __init__(self, tools, db_connection):
        self.tools = tools
        self.db = db_connection
        self.analysis_history = []
    
    def analyze_stock(self, stock_symbol: str, analysis_depth: str = "comprehensive") -> str:
        """
        主分析方法 - Agent自主决定分析策略
        """
        print(f"🤖 交易分析Agent启动，目标股票：{stock_symbol}")
        print(f"📈 分析深度：{analysis_depth}")
        
        # Agent自主规划分析步骤
        plan_prompt = f"""
        你是专业的银行投资分析师，需要分析股票 {stock_symbol}。
        
        可用工具：
        1. get_stock_data(stock_symbol, days) - 获取历史价格数据
        2. calculate_technical_indicators(stock_data) - 计算技术指标
        3. get_market_sentiment(stock_symbol) - 获取市场情绪
        
        请制定分析计划，说明你将如何使用这些工具进行{analysis_depth}分析。
        """
        
        print("🧠 Agent规划分析策略...")
        plan = call_claude_stream(plan_prompt, "", max_tokens=300, temperature=0.3)
        print(f"\n📋 分析计划制定完成\n")
        
        # 执行分析步骤
        return self._execute_analysis(stock_symbol, plan)
    
    def _execute_analysis(self, stock_symbol: str, plan: str) -> str:
        """执行具体的分析步骤"""
        
        # 步骤1：获取股票数据
        print("📊 步骤1：获取股票历史数据...")
        stock_data = self.tools.get_stock_data(stock_symbol, 30)
        print(f"✅ 获取{stock_symbol}近30天数据，当前价格：{stock_data['current_price']}")
        
        # 步骤2：计算技术指标
        print("\n📈 步骤2：计算技术指标...")
        indicators = self.tools.calculate_technical_indicators(stock_data)
        print(f"✅ 技术指标计算完成：RSI={indicators['RSI']}, 趋势={indicators['price_momentum']}")
        
        # 步骤3：市场情绪分析
        print("\n💭 步骤3：分析市场情绪...")
        sentiment = self.tools.get_market_sentiment(stock_symbol)
        print(f"✅ 市场情绪：{sentiment['market_sentiment']}, 机构评级：{sentiment['institutional_rating']}")
        
        # Agent基于数据进行综合分析
        analysis_prompt = f"""
        作为银行投资分析师，请基于以下数据进行综合分析：
        
        股票代码：{stock_symbol}
        当前价格：{stock_data['current_price']}
        
        技术指标：
        - RSI: {indicators['RSI']}
        - 5日均线: {indicators['SMA_5']}
        - 10日均线: {indicators['SMA_10']}
        - 价格趋势: {indicators['price_momentum']}
        - 成交量趋势: {indicators['volume_trend']}
        
        市场情绪：
        - 整体情绪: {sentiment['market_sentiment']}
        - 机构评级: {sentiment['institutional_rating']}
        - 新闻情绪分数: {sentiment['news_sentiment_score']}
        - 分析师目标价: {sentiment['analyst_target_price']}
        
        请提供专业的投资分析报告，包含：
        1. 技术面分析
        2. 市场情绪评估
        3. 风险评价
        4. 投资建议（必须包含免责声明）
        """
        
        print("\n🔍 步骤4：Agent综合分析中...")
        analysis_result = call_claude_stream(
            analysis_prompt, 
            "你是专业银行投资分析师，分析要客观、专业，必须包含风险提示。",
            max_tokens=800, 
            temperature=0.4
        )
        
        # 保存分析结果
        self._save_analysis(stock_symbol, indicators, sentiment, analysis_result)
        
        return analysis_result
    
    def _save_analysis(self, symbol: str, indicators: Dict, sentiment: Dict, result: str):
        """保存分析结果到数据库"""
        cursor = self.db.cursor()
        cursor.execute('''
            INSERT INTO trading_analysis (symbol, analysis_type, indicators, recommendation, confidence_score)
            VALUES (?, ?, ?, ?, ?)
        ''', (
            symbol,
            'comprehensive',
            json.dumps({**indicators, **sentiment}, ensure_ascii=False),
            result[:500],  # 截取前500字符
            0.75  # 示例置信度
        ))
        self.db.commit()
        print("\n💾 分析结果已保存到本地数据库")

# 创建并测试Agent
agent = TradingAnalysisAgent(tools, db_conn)
print("🤖 交易分析Agent已创建，准备开始分析...")


In [ ]:
# 执行Agent分析 - 展示自主决策能力
print("🚀 开始执行Agent分析示例\n" + "="*60)

# 分析银行板块代表股票
bank_stocks = ["000001.SZ", "601398.SH", "000002.SZ"]  # 平安银行、工商银行、万科A(地产)

for stock in bank_stocks[:1]:  # 演示用，只分析第一只
    print(f"\n🎯 正在分析：{stock}")
    result = agent.analyze_stock(stock, "comprehensive")
    print(f"\n✅ {stock} 分析完成\n")
    print("="*60)
    break  # 仅演示一只股票的完整流程

print("\n🎊 Agent演示完成！")
print("💡 关键特点：Agent自主规划 → 动态调用工具 → 基于反馈调整 → 生成专业报告")


In [ ]:
## 🤝 多智能体协作：Orchestrator-Workers模式（2分钟）

### 模式特点
- **Orchestrator（编排者）**：中央LLM负责任务分解和结果聚合
- **Workers（工作者）**：专门的Agent负责具体子任务
- **适用场景**：复杂的、需要多角度分析的金融任务

### 银行业应用实例
1. **投研报告生成**：
   - Worker1: 基本面分析
   - Worker2: 技术面分析
   - Worker3: 风险评估
   - Orchestrator: 统筹并生成最终报告

2. **银行板块全景分析**：
   - Worker1: 大型银行股分析
   - Worker2: 中小银行股分析
   - Worker3: 政策环境影响
   - Orchestrator: 整体板块判断


In [ ]:
# 多智能体协作概念演示

print("🎭 多智能体协作模式展示")
print("="*50)

# 概念性演示：Orchestrator-Workers模式
orchestrator_demo = """
银行板块分析的多智能体协作流程：

📋 Orchestrator（总协调员）规划：
   ├─ 任务分解：大型银行 + 中型银行 + 政策影响
   ├─ 分配Workers：每个Worker专注特定领域
   └─ 聚合决策：统筹所有分析结果

👷‍♂️ Worker1: 大型银行专家（工商银行、建设银行分析）
👷‍♀️ Worker2: 中型银行专家（平安银行、招商银行分析）  
👷‍♂️ Worker3: 政策环境专家（货币政策、监管影响分析）

🎯 最终输出：银行板块综合投资建议报告
"""

print(orchestrator_demo)

# 多智能体的关键优势
advantages = [
    "🔄 专业分工：每个Agent专注擅长领域",
    "⚡ 并行处理：多个分析同时进行",
    "🧠 智能协调：Orchestrator统筹全局",
    "📈 质量提升：多角度分析减少盲点",
    "🔧 可扩展性：容易添加新的专业Worker"
]

print("\n💡 多智能体协作优势：")
for advantage in advantages:
    print(f"   {advantage}")

print(f"\n✅ 多智能体概念演示完成！")
print("📚 实际实现时可根据业务需求逐步构建复杂的协作体系")


In [ ]:
## 📚 课程总结与最佳实践（3分钟）

### 🎯 核心要点回顾

1. **Anthropic三大原则**：
   - **Simplicity**：优先选择最简单的解决方案
   - **Transparency**：显式输出Agent思考过程
   - **Robust ACI**：精心设计Agent-工具接口

2. **Workflow vs Agent选择策略**：
   ```
   简单查询 → 单次LLM调用
   标准流程 → Workflow模式（如资讯摘要）
   复杂决策 → Agent模式（如投资分析）
   多维分析 → Multi-Agent协作
   ```

3. **五大设计模式**：
   - Prompt Chaining: 固定步骤串行
   - Routing: 分类导向专门处理
   - Parallelization: 并行计算和投票
   - Orchestrator-Workers: 动态任务分解
   - Evaluator-Optimizer: 迭代质量改进

### 🏦 银行实施建议

**⚠️ 合规要求**：
- AI仅作"分析工具"，不自动执行交易
- 所有投资建议必须包含免责声明
- 记录完整的分析过程便于审计
- 后端风控系统严格校验所有操作

**🚀 实施路径**：
1. **阶段1**：从资讯摘要等Workflow开始
2. **阶段2**：引入单Agent交易分析
3. **阶段3**：构建多Agent协作体系

### 📈 技术发展趋势

- **Model Context Protocol (MCP)**：统一工具生态
- **更强推理能力**：Agent规划质量持续提升
- **成本下降**：使复杂Agent架构更经济可行

> **记住**：成功的AI Agent系统不在于复杂性，而在于恰到好处地解决实际业务问题！


In [ ]:
# 课程结束：数据清理和学习记录

# 查看本次学习生成的数据
cursor = db_conn.cursor()

print("📊 本次学习生成的分析记录：")
print("\n1. 资讯分析记录：")
cursor.execute("SELECT title, sentiment, risk_level, timestamp FROM news_analysis ORDER BY timestamp DESC LIMIT 3")
news_records = cursor.fetchall()
for record in news_records:
    print(f"   • {record[0][:30]}... | 情绪:{record[1]} | 风险:{record[2]} | {record[3]}")

print("\n2. 交易分析记录：")
cursor.execute("SELECT symbol, analysis_type, confidence_score, timestamp FROM trading_analysis ORDER BY timestamp DESC LIMIT 3")
trading_records = cursor.fetchall()
for record in trading_records:
    print(f"   • {record[0]} | 类型:{record[1]} | 置信度:{record[2]} | {record[3]}")

print(f"\n📈 学习成果统计：")
print(f"   • 掌握了 Workflow vs Agent 的核心区别")
print(f"   • 实践了 5 种主要的Agentic设计模式")
print(f"   • 构建了银行场景的实用AI Agent系统")
print(f"   • 体验了多智能体协作的强大能力")

print(f"\n🎓 恭喜完成 AI Agent Workflow 基础教程！")
print(f"⏰ 总用时：约18分钟")
print(f"🔥 核心掌握：基于Anthropic最佳实践的Agent架构设计")

# 关闭数据库连接
db_conn.close()
print(f"\n💾 本地数据库已保存，可用于后续实验和开发")

print(f"\n🚀 下一步建议：")
print(f"   1. 阅读 Anthropic 官方文档深入学习")
print(f"   2. 在实际银行业务中试点简单的Workflow")
print(f"   3. 逐步探索更复杂的Agent应用场景")
print(f"   4. 关注 MCP 生态发展，扩展工具能力")
